## Data Manupulation by using PySpark


### Get daily revenue by product

In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from pyspark.sql import SQLContext

from awsglue.job import Job

glueContext = GlueContext(SparkContext.getOrCreate())

sqlContext = SQLContext(sc)

from pyspark.sql import *

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
10,application_1570490007969_0011,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Reading different file format

In [ ]:
help(sqlContext.read.orc)

In [ ]:
help(sqlContext.read.text)

##  Get daily revenue by product 

In [2]:
## Open CSV file

## Replace bucket name here with your bucket name

orders = sc.textFile("s3://raw-immersion-day-navnis/retail_db_csv/retail_db_retail_db_orders/version_0/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
type(orders)

In [3]:
### displaying 10 recods 

for i in orders.take(10):
    print(i)
    



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error occurred while calling o89.partitions.
: com.amazon.ws.emr.hadoop.fs.shaded.com.amazonaws.SdkClientException: Unable to execute HTTP request: Connect to raw-immersion-day-navnis.s3.amazonaws.com:443 [raw-immersion-day-navnis.s3.amazonaws.com/52.216.138.187] failed: connect timed out
	at com.amazon.ws.emr.hadoop.fs.shaded.com.amazonaws.http.AmazonHttpClient$RequestExecutor.handleRetryableException(AmazonHttpClient.java:1134)
	at com.amazon.ws.emr.hadoop.fs.shaded.com.amazonaws.http.AmazonHttpClient$RequestExecutor.executeHelper(AmazonHttpClient.java:1080)
	at com.amazon.ws.emr.hadoop.fs.shaded.com.amazonaws.http.AmazonHttpClient$RequestExecutor.doExecute(AmazonHttpClient.java:745)
	at com.amazon.ws.emr.hadoop.fs.shaded.com.amazonaws.http.AmazonHttpClient$RequestExecutor.executeWithTimer(AmazonHttpClient.java:719)
	at com.amazon.ws.emr.hadoop.fs.shaded.com.amazonaws.http.AmazonHttpClient$RequestExecutor.execute(AmazonHttpClient.java:701)
	at com.amazon.ws.emr.hadoop.fs.shaded.co

In [3]:
## Check first record

orders.first()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error occurred while calling o89.partitions.
: com.amazon.ws.emr.hadoop.fs.shaded.com.amazonaws.SdkClientException: Unable to execute HTTP request: Connect to raw-immersion-day-navnis.s3.amazonaws.com:443 [raw-immersion-day-navnis.s3.amazonaws.com/52.216.110.35] failed: connect timed out
	at com.amazon.ws.emr.hadoop.fs.shaded.com.amazonaws.http.AmazonHttpClient$RequestExecutor.handleRetryableException(AmazonHttpClient.java:1134)
	at com.amazon.ws.emr.hadoop.fs.shaded.com.amazonaws.http.AmazonHttpClient$RequestExecutor.executeHelper(AmazonHttpClient.java:1080)
	at com.amazon.ws.emr.hadoop.fs.shaded.com.amazonaws.http.AmazonHttpClient$RequestExecutor.doExecute(AmazonHttpClient.java:745)
	at com.amazon.ws.emr.hadoop.fs.shaded.com.amazonaws.http.AmazonHttpClient$RequestExecutor.executeWithTimer(AmazonHttpClient.java:719)
	at com.amazon.ws.emr.hadoop.fs.shaded.com.amazonaws.http.AmazonHttpClient$RequestExecutor.execute(AmazonHttpClient.java:701)
	at com.amazon.ws.emr.hadoop.fs.shaded.com

In [ ]:
## As first record is header; let's remove it

ordersheader = orders.first()

orderswithoutheader = orders.filter(lambda c: c != ordersheader)

In [ ]:
## After removing header

orderswithoutheader.first()

In [ ]:
## string manipulation

s = '"2013-07-25 00:00:00.0",PROCESSING,9198,20'

In [ ]:
## checking type

type(s)

In [ ]:
## Convert string into list

o = s.split(",")

type(o)

In [ ]:
o[1]

In [ ]:
for i in orderswithoutheader.take(10):
    print(i)

In [ ]:
## Find distinct order status

ordersstatus = orderswithoutheader.map(lambda o : o.split(',')[1]).distinct()

In [ ]:
ordersstatus.take(20)

In [ ]:
## Filter all the orders with status = COMPLETE , CLOSED

cc_orders = orderswithoutheader.filter(lambda o : o.split(',')[1] in ('COMPLETE', 'CLOSED'))

In [ ]:
for i in cc_orders.take(10):
    print(i)

In [ ]:
## Reading order_items files


order_items = sc.textFile('s3://raw-immersion-day-navnis/retail_db_csv/retail_db_retail_db_order_items/version_0/')

In [ ]:
order_items.take(10)

In [ ]:
order_items_header = order_items.first()

In [ ]:
order_items_withoutheader = order_items.filter(lambda oi : oi != order_items_header)

In [ ]:
for i in order_items_withoutheader.take(10):
    print(i)

### JOIN between Orders and Order Item

#### Orders need order_id and order_date
#### Orders_item need order_item_id , order_item_product_id , order_item_subtotal

In [ ]:
## In order to perform join in pyspark, object should be in key,value format

## (K,V1)  (K,V2) ==> Join ==> ( K , (V1,V2))

cc_ordersMap = cc_orders.map(lambda o : (int(o.split(",")[3]), o.split(",")[0]))

order_items_cc = order_items_withoutheader.map(lambda oi : (int(oi.split(",")[0]), (int(oi.split(",")[1]) , float(oi.split(",")[4]))))

In [ ]:
for i in cc_ordersMap.take(10):
    print(i)

In [ ]:
for i in order_items_cc.take(10):
    print(i)

In [ ]:
## JOIN

cc_ordersJoin = cc_ordersMap.join(order_items_cc)

In [ ]:
for i in cc_ordersJoin.take(10):
    print(i)

### Daily revenue per product id

In [ ]:
## Daily revenue per product id

## (80, ('"2013-07-25 00:00:00.0"', (957, 299.9800109863281))) = (('"2013-07-25 00:00:00.0"', 957),80)

In [ ]:
t = (80, ('"2013-07-25 00:00:00.0"', (957, 299.9800109863281)))

In [ ]:
t[1][0] , t[1][1][0] ,  t[1][1][1] 

In [ ]:
cc_ordersJoinMap = cc_ordersJoin.map(lambda t : ((t[1][0] , t[1][1][0]), t[1][1][1] ))

In [ ]:
for i in cc_ordersJoinMap.take(10):
    print(i)

### aggregate function  

In [ ]:
from operator import add

In [ ]:
dailyRevenuePerProductId = cc_ordersJoinMap.reduceByKey(add)

In [ ]:
for i in dailyRevenuePerProductId.take(10):
    print(i)

In [ ]:
## Reading product files

products = sc.textFile('s3://raw-immersion-day-navnis/retail_db_csv/retail_db_retail_db_products/version_0/')

In [ ]:
for i in products.take(10):
    print(i)

In [ ]:
productheader = products.first()

In [ ]:
productwithoutheader = products.filter(lambda p : p != productheader)

In [ ]:
productsMap = productwithoutheader.map(lambda p : (int(p.split(",")[1]),p.split(",")[4]))

In [ ]:
for i in productsMap.take(5):
    print(i)

In [ ]:
## Convert dailyRevenuePerProductId 

dailyRevenuePerProductIdMap = dailyRevenuePerProductId.map(lambda d : (d[0][1], (d[0][0],d[1])))

In [ ]:
for i in dailyRevenuePerProductIdMap.take(5):
    print(i)

In [ ]:
dailyRevenuePerProductIdjoin = dailyRevenuePerProductIdMap.join(productsMap)

In [ ]:
for i in dailyRevenuePerProductIdjoin.take(5):
    print(i)

In [ ]:
t = (365, (('"2013-10-20 00:00:00.0"', 2039.660057067871), '"Perfect Fitness Perfect Rip Deck"'))

In [ ]:
t[1][0][0], t[1][0][1]

In [ ]:
dailyRevenuePerProduct = dailyRevenuePerProductIdjoin.map(lambda t : ((t[1][0][0], -t[1][0][1]), t[1][0][0] + "," + \
                                                                     str(t[1][0][1]) + "," + t[1][1]))

In [ ]:
for i in dailyRevenuePerProduct.take(5):
    print(i)

### Sort function

In [ ]:
dailyRevenuePerProductSorted = dailyRevenuePerProduct.sortByKey()

In [ ]:
for i in dailyRevenuePerProductSorted.take(10):
    print(i)

### Final Result 

In [ ]:
dailyRevenuePerProductResult = dailyRevenuePerProductSorted.map(lambda r : r[1])

In [ ]:
for i in dailyRevenuePerProductResult.take(10):
    print(i)

In [ ]:
from pyspark.sql import Row

### Convert RDD into Data Frame

In [ ]:
## Convert RDD into Data Frame; it will allow file to save as parquet 

dailyRevenuePerProductResultDF = dailyRevenuePerProductResult.map(lambda o :Row(date = str(o.split(",")[0]), \
                                                                                 product = str(o.split(",")[2]), \
                                                                                revenue = str(o.split(",")[1]))).toDF()

In [ ]:
type(dailyRevenuePerProductResultDF)

In [ ]:
dailyRevenuePerProductResultDF.show()

## Save file as Parquet



In [ ]:
sqlContext.setConf("spark.sql.parquet.compression.codec","gzip")

dailyRevenuePerProductResultDF.write.parquet('s3://raw-immersion-day-navnis/dailyRevenueByProductResultParquet/')

### After saving data as Parquet file; create dailyRevenuePerProduct table in Athena

### Please bucket name with s3://raw-immersion-day-navnis/dailyRevenueByProductResultParquet

    Create database dailyrevenue;

    CREATE EXTERNAL TABLE `dailyrevenueperproduct`(
      `date` string, 
      `product` string, 
      `revenue` string)
    ROW FORMAT SERDE 
      'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe' 
    STORED AS INPUTFORMAT 
      'org.apache.hadoop.mapred.TextInputFormat' 
    OUTPUTFORMAT 
      'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
    LOCATION
      's3://raw-immersion-day-navnis/dailyRevenueByProductResultParquet'
    TBLPROPERTIES (
      'has_encrypted_data'='false', 
      'transient_lastDdlTime'='1570611539')


## Data Manupulation by using SparkSQL

### Perform count by status

## Using sparkSQL

In [2]:
## reading data from retail_db_retail_db_orders

## please grant select access to table "retail_db_retail_db_orders" to "Glue_Role_Admin_Policy"

orders = glueContext.create_dynamic_frame.from_catalog(database="retail_db", table_name="retail_db_retail_db_orders")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error occurred while calling o89.getDynamicFrame.
: com.amazon.ws.emr.hadoop.fs.shaded.com.amazonaws.SdkClientException: Unable to execute HTTP request: Connect to raw-immersion-day-navnis.s3.amazonaws.com:443 [raw-immersion-day-navnis.s3.amazonaws.com/52.216.101.163] failed: connect timed out
	at com.amazon.ws.emr.hadoop.fs.shaded.com.amazonaws.http.AmazonHttpClient$RequestExecutor.handleRetryableException(AmazonHttpClient.java:1134)
	at com.amazon.ws.emr.hadoop.fs.shaded.com.amazonaws.http.AmazonHttpClient$RequestExecutor.executeHelper(AmazonHttpClient.java:1080)
	at com.amazon.ws.emr.hadoop.fs.shaded.com.amazonaws.http.AmazonHttpClient$RequestExecutor.doExecute(AmazonHttpClient.java:745)
	at com.amazon.ws.emr.hadoop.fs.shaded.com.amazonaws.http.AmazonHttpClient$RequestExecutor.executeWithTimer(AmazonHttpClient.java:719)
	at com.amazon.ws.emr.hadoop.fs.shaded.com.amazonaws.http.AmazonHttpClient$RequestExecutor.execute(AmazonHttpClient.java:701)
	at com.amazon.ws.emr.hadoop.fs.shad

In [ ]:
type(orders)

In [ ]:
## to check count in table

orders.count()

In [ ]:
## to check schema

orders.printSchema()

In [ ]:
### Convert into dataframe

orders = orders.toDF()

In [ ]:
orders.show(10)

#### if you check records in above data frame you see columns name are in result

In [ ]:
type(orders)

In [ ]:
### Convert data into RDD

orders = orders.rdd

In [ ]:
type(orders)

In [ ]:
## check first record

ordersheader = orders.first()

In [ ]:
type(ordersheader)

In [ ]:
### Filter first record 

orderwithoutheader = orders.filter(lambda o : o != ordersheader )

In [ ]:
for i in orderwithoutheader.take(10):
    print(i)

In [ ]:
from pyspark.sql import *

In [ ]:
orderwithoutheader = orderwithoutheader.map(lambda o :Row(order_date = o[0], \
                                                         order_status = o[1], \
                                                        order_customer_id = o[2], \
                                                        order_id = o[3])).toDF()

In [ ]:
orderwithoutheader.show(10)

In [ ]:
### create temporary table from data frame

orderwithoutheader.createOrReplaceTempView("t_orders")

In [ ]:
## First row is showing as header of table

spark.sql("select * from t_orders limit 10").show()

In [ ]:
spark.sql("select order_status , count(order_status) as status_count from t_orders \
            group by order_status order by status_count desc").show()